In [0]:
# Define schema for input files
jsonSchema = StructType([
  StructField("key1", StringType()),
  StructField("key2", StructType([
    StructField("subkey1", IntegerType()),
    StructField("subkey2", StringType())
  ]))
])

# Stream json files to delta table
(spark.readStream.format("cloudFiles")
  .option("cloudFiles.format", "json")
  .option("cloudFiles.schemaLocation", "/path/_checkpoint/address_autoload/")
  .option("cloudFiles.useNotifications",True) # Provide values for below
  .option("cloudFiles.clientId", "xxxxxxxxxxxxxxxxxx") # ID Service Principal
  .option("cloudFiles.clientSecret", "xxxxxxxxxxxxxxxxxx") # Secret value
  .option("cloudFiles.tenantId", "xxxxxxxxxxxxxxxxxx") # Tenant ID
  .option("cloudFiles.subscriptionId", "xxxxxxxxxxxxxxxxxx") # Subscription ID
  .option("cloudFiles.connectionString",
  "DefaultEndpointsProtocol=https; \
  AccountName=<storage-account>; \
  AccountKey=<account-key>;EndpointSuffix=core.windows.net")
  .option("cloudFiles.resourceGroup", "xxxxxxxxxxxxxxxxxx") # RG Name
  .schema(jsonSchema)
  .load("/path/to/json/files")
  .writeStream
  .format("delta")
  .option("checkpointLocation", "/path/_checkpoint/address_autoload/")
  .toTable("bronze_adventureworks.address")
)